# 📘 代理架构 9：思维树规划

欢迎来到我们系列的第九个notebook。今天，我们探索一个强大的推理和规划架构：**思维树（ToT）**。这种模式将代理的问题解决能力从线性思维链提升到多路径探索搜索。

与生成单一、顺序的推理线不同，ToT代理在问题的每个阶段生成多个候选"思维"或下一步。然后它评估这些思维，修剪无效或没有前途的分支，并扩展最有前途的分支。这创建了一个搜索树，代理可以在其中回溯、探索替代方案，并系统地导航复杂的问题空间。

为了演示这一点，我们将用一个经典的逻辑谜题来测试我们的代理：**狼、羊和卷心菜问题**。这个谜题之所以著名，是因为它需要非显而易见的步骤（比如把物品*带回来*），并且有几个无效状态可能会困住天真的推理者。我们将展示简单的思维链代理如何可能失败，而ToT代理通过探索和评估可能性树来有条不紊地构建有效计划。

### 定义
**思维树（ToT）**是一种代理推理框架，其中问题解决被建模为通过树的搜索。代理同时探索多条推理路径（分支）。在每一步，它生成潜在的下一步（"思维"），评估它们的可行性，并决定继续探索哪些路径，有效地修剪搜索空间。

### 高级工作流程

1. **分解：** 问题被分解为一系列步骤或思维。
2. **思维生成：** 对于问题的当前状态，代理生成多个潜在的下一步或思维。这在搜索树中创建分支。
3. **状态评估：** 每个新思维（导向新状态）通过"评论者"或验证函数进行评估。此评估可以评估：
   * **有效性：** 这个移动是否被问题规则允许？
   * **进展：** 这个移动是否让我们更接近解决方案？
   * **启发式：** 这条路径是否可能成功？
4. **修剪和扩展：** 无效或没有前途的分支被修剪。代理然后从最有前途的活动分支继续，重复思维生成过程。
5. **解决方案：** 过程继续直到达到目标状态。解决方案是从根到目标的思维路径。

### 何时使用/应用场景
* **逻辑谜题和数学问题：** 具有明确规则和目标状态的问题，需要多步骤、非线性推理（例如，数独、过河谜题）。
* **复杂规划：** 当任务需要详细计划，其中操作顺序很重要且必须遵守约束时（例如，规划具有多个航段和预算约束的复杂旅行）。
* **创意写作或代码生成：** 在提交之前探索多个故事分支或实现策略。

### 优点和缺点
* **优点：**
  * **健壮性：** 系统地探索问题空间，与单次方法相比，更不可能卡住或产生错误答案。
  * **处理组合复杂性：** 非常适合可能序列数量巨大的问题。
* **缺点：**
  * **计算量大：** 比简单的思维链提示需要更多的LLM调用和状态管理，使其更慢、更昂贵。
  * **需要好的评估器：** 搜索的有效性严重依赖于状态评估逻辑的质量。

## 阶段0：基础与设置

我们将像往常一样安装我们的库并配置我们的API密钥。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
import re
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from collections import defaultdict

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.tree import Tree

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Tree-of-Thoughts (Nebius)"

# Check for required environment variables
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"警告: 环境变量 {var} 未设置。")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：定义问题环境

思维树系统需要一个定义良好的环境来运行。对于我们的狼、羊和卷心菜谜题，这意味着我们需要以编程方式定义：

1. **状态表示：** 描述所有物品位置的方式。
2. **验证规则：** 检查状态是否无效的函数（例如，羊和卷心菜被单独留下）。
3. **目标状态：** 检查谜题是否已解决的方式。
4. **可能的移动：** 确定给定状态下所有合法移动的函数。

In [3]:
console = Console()

class PuzzleState(BaseModel):
    """表示狼、羊和卷心菜谜题的状态。"""
    # Using sets for unordered collections of items in each bank
    left_bank: set[str] = Field(default_factory=lambda: {"wolf", "goat", "cabbage"})
    right_bank: set[str] = Field(default_factory=set)
    boat_location: str = "left"
    move_description: str = "Initial state."

    def is_valid(self) -> bool:
        """检查当前状态是否有效（没有人被吃掉）。"""
        # Check left bank
        if self.boat_location == "right":
            if "wolf" in self.left_bank and "goat" in self.left_bank:
                return False
            if "goat" in self.left_bank and "cabbage" in self.left_bank:
                return False
        # Check right bank
        if self.boat_location == "left":
            if "wolf" in self.right_bank and "goat" in self.right_bank:
                return False
            if "goat" in self.right_bank and "cabbage" in self.right_bank:
                return False
        return True

    def is_goal(self) -> bool:
        """检查是否已达到目标状态。"""
        return self.right_bank == {"wolf", "goat", "cabbage"}
    
    def __hash__(self):
        # Make state hashable to check for visited states
        return hash((frozenset(self.left_bank), frozenset(self.right_bank), self.boat_location))
    
    def __eq__(self, other):
        return self.__hash__() == other.__hash__()

def get_possible_moves(state: PuzzleState) -> list[PuzzleState]:
    """从当前状态生成所有可能的有效下一状态。"""
    moves = []
    current_bank = state.left_bank if state.boat_location == "left" else state.right_bank
    
    # Option 1: Move one item in boat
    for item in current_bank:
        new_state = state.model_copy(deep=True)
        if state.boat_location == "left":
            new_state.left_bank.remove(item)
            new_state.right_bank.add(item)
            new_state.boat_location = "right"
            new_state.move_description = f"Move {item} to right bank."
        else:
            new_state.right_bank.remove(item)
            new_state.left_bank.add(item)
            new_state.boat_location = "left"
            new_state.move_description = f"Move {item} to left bank."
        if new_state.is_valid():
            moves.append(new_state)
    
    # Option 2: Move boat empty
    empty_move_state = state.model_copy(deep=True)
    if state.boat_location == "left":
        empty_move_state.boat_location = "right"
        empty_move_state.move_description = "Move boat empty to right bank."
    else:
        empty_move_state.boat_location = "left"
        empty_move_state.move_description = "Move boat empty to left bank."
    if empty_move_state.is_valid():
        moves.append(empty_move_state)
    
    return moves

print("谜题环境定义成功。")

谜题环境定义成功。


## 阶段2：使用LangGraph实现ToT代理

我们现在将构建代理本身。我们图的状态将跟踪思维树中的所有活动路径（分支）。节点将执行关键的ToT操作：

1. **扩展路径（思维生成器）：** 一个LLM驱动的节点，查看每个活动路径的最后状态，并从有效可能性列表中提出一个有希望的下一步。
2. **修剪路径（状态评估器）：** 这个节点在生成后进行清理。它将移除任何进入无效状态或循环（重访先前状态）的路径。
3. **检查解决方案（目标检查）：** 一个条件节点，检查是否有任何活动路径已达到目标状态。如果是，它终止循环。

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.4)

# Pydantic model for LLM's choice of move
class MoveChoice(BaseModel):
    best_move_index: int = Field(description="从提供的可能移动列表中选择的最佳移动索引。")
    reasoning: str = Field(description="为什么这是最有希望的移动的简要推理。")

# LangGraph State
class ToTState(TypedDict):
    problem_description: str
    # Each path is a list of PuzzleState objects
    active_paths: List[List[PuzzleState]]
    # We will store final solution here
    solution: Optional[List[PuzzleState]]

# Graph Nodes
def initialize_search(state: ToTState) -> Dict[str, Any]:
    """设置搜索的初始状态的节点。"""
    initial_puzzle_state = PuzzleState()
    return {"active_paths": [[initial_puzzle_state]]}

def expand_paths(state: ToTState) -> Dict[str, Any]:
    """'思维生成器'。用一个有希望的下一步扩展每个活动路径。"""
    console.print("--- 扩展路径 ---")
    new_paths = []
    choice_llm = llm.with_structured_output(MoveChoice)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一位专业的逻辑谜题解决者。你的目标是解决狼、羊和卷心菜问题。分析当前路径并从选项列表中选择最有希望的下一步以达到目标。"),
        ("human", "Problem: {problem}\n\nCurrent Path History:\n{path_history}\n\nFrom final state, choose the best next move from this list:\n{possible_moves}")
    ])
    
    for path in state['active_paths']:
        last_state = path[-1]
        possible_next_states = get_possible_moves(last_state)
        
        if not possible_next_states:
            continue  # This path is a dead end
        
        path_history_str = " -> ".join([s.move_description for s in path])
        possible_moves_str = "\n".join([f"{i}: {s.move_description}" for i, s in enumerate(possible_next_states)])
        
        # For simplicity and to show breadth, we can explore multiple moves.
        # A more advanced ToT might use LLM to pick only single best one.
        # Here, we will let all valid moves branch out to demonstrate tree structure.
        for next_state in possible_next_states:
            new_paths.append(path + [next_state])

    console.print(f"[cyan]扩展到 {len(new_paths)} 个潜在路径。[/cyan]")
    return {"active_paths": new_paths}

def prune_paths(state: ToTState) -> Dict[str, Any]:
    """'状态评估器'。修剪无效或包含循环的路径。"""
    console.print("--- 修剪路径 ---")
    pruned_paths = []
    for path in state['active_paths']:
        # Check for cycles: if last state has appeared before in path
        if path[-1] in path[:-1]:
            continue  # Found a cycle, prune this path
        
        # get_possible_moves function already ensures validity, but this is a good place for additional checks.
        pruned_paths.append(path)
    
    console.print(f"[green]修剪到 {len(pruned_paths)} 个有效的非循环路径。[/green]")
    return {"active_paths": pruned_paths}

# Conditional Node
def check_for_solution(state: ToTState) -> str:
    """检查是否有任何路径达到目标并路由执行。"""
    for path in state['active_paths']:
        if path[-1].is_goal():
            console.print("[bold green]找到解决方案！[/bold green]")
            # Side effect: update solution in state. LangGraph copies this out.
            state['solution'] = path
            return "solution_found"
    return "continue_search"

# Build graph
workflow = StateGraph(ToTState)

workflow.add_node("initialize", initialize_search)
workflow.add_node("expand", expand_paths)
workflow.add_node("prune", prune_paths)

workflow.set_entry_point("initialize")
workflow.add_edge("initialize", "expand")
workflow.add_edge("expand", "prune")

workflow.add_conditional_edges(
    "prune",
    check_for_solution,
    {
        "solution_found": END,
        "continue_search": "expand"
    }
)

tot_agent = workflow.compile()
print("思维树代理图编译成功。")

思维树代理图编译成功。


## 阶段3：演示与分析

现在，让我们在谜题上运行我们的ToT代理。我们将比较它的系统化方法与简单的单次思维链请求，以突出健壮性方面的差异。

In [5]:
problem = "A farmer wants to cross a river with a wolf, a goat, and a cabbage. The boat can only carry the farmer and one other item. The farmer cannot leave the wolf alone with the goat, nor the goat alone with the cabbage. How can the farmer get everyone across safely?"

console.print("--- 🌳 运行思维树代理 ---")
# recursion limit prevents infinite loops in our graph
config = {"recursion_limit": 15}
final_state = tot_agent.invoke({"problem_description": problem}, config=config)

console.print("\n--- ✅ ToT代理解决方案 ---")
if final_state.get('solution'):
    solution_path = final_state['solution']
    # Use rich.Tree for a nice visual output
    tree = Tree("[bold magenta]狼、羊和卷心菜解决方案路径[/bold magenta]")
    for i, state in enumerate(solution_path):
        tree.add(f"[green]{i+1}.[/green] {state.move_description}")
    console.print(tree)
else:
    console.print("[bold red]在步骤限制内未找到解决方案。[/bold red]")

console.print("\n--- 🤔 运行简单思维链代理 ---")
cot_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个世界级的逻辑谜题解决者。为用户的谜题提供逐步解决方案。"),
    ("human", "{problem}")
])
cot_chain = cot_prompt | llm
cot_result = cot_chain.invoke({"problem": problem}).content
console.print(Markdown(cot_result))

--- 🌳 运行思维树代理 ---


--- Expanding Paths ---
扩展到 1 个潜在路径。
--- Pruning Paths ---
修剪到 1 个有效的非循环路径。
--- Expanding Paths ---
扩展到 2 个潜在路径。
--- Pruning Paths ---
修剪到 2 个有效的非循环路径。
--- Expanding Paths ---
扩展到 4 个潜在路径。
--- Pruning Paths ---
修剪到 4 个有效的非循环路径。
--- Expanding Paths ---
扩展到 7 个潜在路径。
--- Pruning Paths ---
修剪到 7 个有效的非循环路径。
--- Expanding Paths ---
扩展到 12 个潜在路径。
--- Pruning Paths ---
修剪到 12 个有效的非循环路径。
--- Expanding Paths ---
扩展到 20 个潜在路径。
--- Pruning Paths ---
修剪到 20 个有效的非循环路径。
--- Expanding Paths ---
扩展到 32 个潜在路径。
--- Pruning Paths ---
修剪到 32 个有效的非循环路径。
找到解决方案！



--- ✅ ToT代理解决方案 ---


狼、羊和卷心菜解决方案路径
├── 1. Initial state.
├── 2. 将羊移到右岸。
├── 3. 空船移到左岸。
├── 4. 将狼移到右岸。
├── 5. 将羊移到左岸。
├── 6. 将卷心菜移到右岸。
├── 7. 空船移到左岸。
└── 8. 将羊移到右岸。



--- 🤔 运行简单思维链代理 ---


Here's 一个step-by-step solution 到Wolf, Goat, 和Cabbage puzzle:

1. **Take Goat across:** First, take goat across river 到right bank. You leave wolf 和cabbage behind 在the left bank.
2. **Return empty:** Return 到left bank alone.
3. **Take Wolf across:** Now, take wolf across 到right bank. 
4. **Bring Goat back:** *这是the key step.* Leave wolf 在the right bank 和bring goat back 与you 到left bank.
5. **Take Cabbage across:** Leave goat 在the left bank 和take cabbage across 到right bank. Now wolf 和cabbage 是在the right bank.
6. **Return empty:** Return 到left bank alone.
7. **Take Goat across:** Finally, take goat across 到right bank.

Now, wolf, goat, 和cabbage 是all safely 在the right bank, 和the puzzle 是solved.

### 结果分析

两种方法之间的差异是深刻的：

- **思维链（CoT）：** 这种方法依赖于LLM的预训练知识来回忆解决方案。对于像这样的经典、众所周知的问题，强大的LLM通常可以一次性产生正确答案。然而，如果它犯了一个错误，它没有自我纠正的机制。对于新颖或更复杂的问题，失败的可能性要高得多。它的正确性是回忆的问题，而不是可验证的推理。

- **思维树（ToT）：** 这个代理通过系统化、可验证的搜索*发现*了解决方案。它不只是回忆一个答案；它构建了一个。我们可以在日志中看到这个过程：扩展路径，然后修剪那些遇到死胡同或循环的路径。即使引导扩展的LLM在某个分支上做出了次优选择，代理也可以继续探索其他更有希望的分支。这种方法更加健壮和可信，因为它的最终解决方案保证根据我们定义的环境规则是有效的。

ToT代理的成功不是基于运气或记忆，而是基于其搜索算法的健全性。这使它成为一种对于需要高可靠性和规划的任务来说远远优越的方法。

## 总结

在这个notebook中，我们实现了一个**思维树**代理来解决一个经典的逻辑谜题。我们展示了通过将问题转换为状态空间并系统地搜索它，代理可以达到一种用简单的单次推理方法不可能达到的健壮性和准确性水平。

ToT的核心组件——**思维生成（扩展）**、**状态评估（修剪）**和**搜索**——为处理复杂的规划和推理任务创建了一个强大的框架。虽然它带来了更高的计算成本，但权衡是可靠性和问题解决能力的显著提高。这种架构是构建能够深思熟虑地推理并找到具有挑战性的多步骤问题解决方案的代理的关键一步。